# CNN

## Libraries

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow.keras as keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix 

%config InlineBackend.figure_format = 'retina'

In [ ]:
# Load training (augmented or normal) and testing data
X_train_aug = np.load('X_train_augmented.npy')
X_test_aug = np.load('X_test.npy')

y_train_aug = np.load('y_train_augmented.npy')
y_test_aug = np.load('y_test.npy')


# CNN Model

In [ ]:
model = keras.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation="relu", input_shape=(16, 15, 1)))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation="relu"))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation="softmax"))

model.compile(
    optimizer="adam",
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)


In [ ]:
# Show model structure
model.summary()

## Compile and train

In [ ]:
history_augmented = model.fit(X_train_augmented.reshape(X_train_augmented.shape[0], 16, 15, 1), y_train_augmented, batch_size=100, epochs=12, shuffle=True, validation_data=(X_test, y_test))

In [ ]:
test_loss_augmented, test_acc_augmented = model.evaluate(X_test,  y_test, verbose=2)

In [ ]:
plt.plot(history_augmented.history['accuracy'], label='accuracy')
plt.plot(history_augmented.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

In [ ]:
plt.plot(history_augmented.history['loss'], label='loss')
plt.plot(history_augmented.history['val_loss'], label = 'val_loss')
plt.ylim([0, 1])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='lower right')